In [1]:
# pip install -U scikit-learn
# pip install matplotlib
# pip install Pillow
# pip install seaborn
# pip install opencv-python
# pip install mtcnn
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import  Dense, Flatten, Conv2D, Rescaling, BatchNormalization, MaxPooling2D, Dropout
import matplotlib.pyplot as plt
import os
import pickle
from glob import glob
from tqdm import tqdm
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import random
import cv2
from mtcnn.mtcnn import MTCNN
import sys

2023-06-01 16:53:50.161374: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


AttributeError: module 'tensorflow.compat.v2.__internal__' has no attribute 'dispatch'

In [2]:
FOLDER_IMAGES = './datasets/dmd/binary_labels'
INPUT_SHAPE = (180, 180, 3)
BATCH_SIZE = 32
IMG_SIZE = (180, 180)
VAL_SPLIT = 0.2
MODEL_SAVE_FOLDER = './models/cnn/cnn_simple/'
EPOCHS = 30

In [3]:
def show_random_images_from_folder(folder_path, num_images=5):
    image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)
                   if os.path.isfile(os.path.join(folder_path, filename))]
    random.shuffle(image_paths)
    selected_image_paths = image_paths[:num_images]
    num_cols = min(num_images, 5)
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))
    axes = np.reshape(axes, (num_rows, num_cols))  # Reshape axes to have desired rows and columns
    for i, image_path in enumerate(selected_image_paths):
        img = Image.open(image_path)
        ax = axes[i // num_cols, i % num_cols]
        ax.imshow(img)
        ax.axis("off")
    if num_images < num_rows * num_cols:
        for i in range(num_images, num_rows * num_cols):
            fig.delaxes(axes[i // num_cols, i % num_cols])
    fig.tight_layout()
    plt.show()

def show_images_from_dataframe(data_frame, num_images = 5):
    for images, labels in data_frame(num_images):
        numpy_images = images.numpy()
        for image in numpy_images:
            plt.imshow(image.astype("uint8"))  # Convert back to uint8 for visualization
            plt.axis("off")
            plt.show()

def crop_face_from_image(image):
    detector = MTCNN()
    faces = detector.detect_faces(image)
    highlighted_image = image.copy()
    ax = plt.gca()
    y_min = float('inf')
    y_max = float('-inf')
    x_min = float('inf')
    x_max = float('-inf')
    for face in faces:
        x, y, width, height = face['box']
        face_border = Rectangle((x, y), width, height, fill=False, color='red')
        ax.add_patch(face_border)
        y_min = min(y_min, y)
        y_max = max(y_max, y + height)
        x_min = min(x_min, x)
        x_max = max(x_max, x + width)
    plt.axis("off")
    plt.show()
    if y_min == float('inf') or y_max == float('-inf') or x_min == float('inf') or x_max == float('-inf'):
        return highlighted_image
    y_min = max(0, y_min)
    y_max = min(image.shape[0], y_max)
    x_min = max(0, x_min)
    x_max = min(image.shape[1], x_max)
    cropped_image = highlighted_image[int(y_min):int(y_max), int(x_min):int(x_max)]
    plt.imshow(cropped_image.astype("uint8"))
    return cropped_image

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  FOLDER_IMAGES,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  FOLDER_IMAGES,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

AttributeError: module 'tensorflow.keras.utils' has no attribute 'image_dataset_from_directory'

In [ ]:
if os.path.exists(MODEL_SAVE_FOLDER):
    model = keras.models.load_model(MODEL_SAVE_FOLDER + 'best.h5')
else:
    model = keras.models.Sequential([
        #normalization
        Rescaling(1./255, input_shape=INPUT_SHAPE),

        Conv2D(filters=32, kernel_size=5, strides=1, activation='relu'),
        Conv2D(filters=32, kernel_size=5, strides=1, activation='relu', use_bias=False),
        BatchNormalization(),
        MaxPooling2D(strides=2),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'),
        Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', use_bias=False),
        BatchNormalization(),
        MaxPooling2D(strides=2),
        Dropout(0.3),

        Flatten(),
        Dense(units=256, activation='relu', use_bias=False),
        BatchNormalization(),

        Dense(units=128, use_bias=False, activation='relu'),

        Dense(units=84, use_bias=False, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(num_classes, name="outputs")
    ])


    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3)

    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=MODEL_SAVE_FOLDER + 'best.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        batch_size=32,
        callbacks=[model_checkpoint]
    )

    with open(MODEL_SAVE_FOLDER + 'model_history.pkl', 'wb') as file:
        pickle.dump(history.history, file)

with open(MODEL_SAVE_FOLDER + 'model_history.pkl', 'rb') as file:
    history = pickle.load(file)
    print(history)

In [ ]:
best_model = load_model(MODEL_SAVE_FOLDER + 'best.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open(MODEL_SAVE_FOLDER + 'best.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
acc = history['accuracy']
val_acc = history['val_accuracy']

loss = history['loss']
val_loss = history['val_loss']

epochs_range = range(30)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Select 5 random images
random_images = random.sample(glob('./datasets/dmd/binary_labels_test/*.jpg'), 5)

# Create a grid of subplots
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i, image_path in enumerate(random_images):
    img = tf.keras.utils.load_img(image_path, target_size=IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    ax = axes[i]
    ax.imshow(img)
    ax.set_title("Predicted: {} (Confidence: {:.2f}%)".format(class_names[np.argmax(score)], 100 * np.max(score)))
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))


# Obtain predictions for the test data
predictions = model.predict(val_ds)
predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_test, predicted_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)